In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *
from executor import *
from dfs import *
from dfs_viewer import *
from main import reset_grid
from melee import *

In [2]:
# MOVE (7, 3) to (5, 5)
# MOVE (3, 2) to (3, 4)
# BOULDER SHELL at (5, 1)
# MOVE (5, 3) to (7, 3)
# REGULAR SHELL at (7, 1)
# CLUSTER SHELL at (3, 2)

In [3]:
# Game(reset_grid(), True)

In [4]:
# grid = reset_grid()

In [5]:
def reset_grid3():
    gb = GridBuilder()

    gb.place(WaterTile(), (0, 0))
    gb.place(WaterTile(), (0, 1))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (0, 2))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (0, 3))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (0, 6))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (0, 7))
    
    
    gb.place(WaterTile(), (1, 0))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (1, 2))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (1, 3))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (1, 6))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (1, 7))
    
    

    gb.place(Destructable(MountainTile(), GroundTile(), 3), (4, 0))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (4, 1))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (4, 6))
    gb.place(WaterTile(), (4, 7))
    
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (5, 1))
    gb.place(Destructable(CivilianTile(), GroundTile(), 2), (5, 6))
    gb.place(WaterTile(), (5, 7))
    
    gb.place(WaterTile(), (6, 0))
    gb.place(Destructable(MountainTile(), GroundTile(), 3), (6, 1))
    gb.place(WaterTile(), (7, 0))
    
    siege_mech = Mech("Siege Mech", 5, 5, 3)
    artillery_mech = Mech("Artillery Mech", 5, 5, 4)
    boulder_mech = Mech("Boulder Mech", 5, 5, 4)

    firefly = Vek("Firefly", 4, 2, 2)
    alpha_firefly = Vek("Alpha Firefly", 6, 2, 2)
    
    hornet = Vek("Hornet", 2, 2, 5, True)
    alpha_hornet = Vek("Alpha Hornet", 4, 4, 5, True)
    
    scarab = Vek("Scarab", 3, 3)
    alpha_scarab = Vek("Scarab", 4, 4)
    
    beetle = Vek("Beetle", 4, 4, 2)
    
    blobber = Vek("Blobber", 3, 3, 2)
    blob = Unit("Blob", 1, 1, 0)
    
    gb.place_on_tile(siege_mech, (3, 2))
    gb.place_on_tile(boulder_mech, (2, 3))
    gb.place_on_tile(artillery_mech, (2, 5))
    
    gb.place_on_tile(alpha_firefly, (5, 4))
    gb.place_on_tile(scarab, (6, 6))
    gb.place_on_tile(alpha_scarab, (6, 2))
    gb.place_on_tile(beetle, (6, 3))

    grid = gb.to_grid()

    for mech in grid.mechs:
        mech.add(Move(mech, grid))
        mech.add(Repair(mech))

    for vek in grid.veks:
        vek.add(Move(vek, grid))

    siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))
    boulder_mech.add(Artillery(boulder_mech, grid, BoulderShell, 3))
    artillery_mech.add(Artillery(artillery_mech, grid, RegularShell, 1))
    
    grid.end_commands = [
        DamageCommand(grid, (1,5), 5),
        DamageCommand(grid, (3,2), 5),
        DamageCommand(grid, (5,5), 5),
        DamageCommand(grid, (6,1), 5),
        VekBeam(alpha_firefly, grid, Compass.EAST, 3),
#         VekCharge(alpha_firefly, grid, Compass.EAST, 3),
        VekShell(scarab, grid, 1, (6, 5)),
        VekShell(alpha_scarab, grid, 3, (1, 2)),
        VekCharge(beetle, grid, Compass.NORTH, 1),
        DamageCommand(grid, (6,4), 1),
        DamageCommand(grid, (5,2), 1),
        DamageCommand(grid, (7,5), 1),
    ]

    return grid

In [6]:
# Game(reset_grid2(), True)

In [ ]:
grid = reset_grid3()
display(grid.show())
dfs = DFS(grid, 7)

,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,🟢,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,🟢,Artillery Mech ♡♡♡♡♡,🟢,🟢
3,🟢,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡♡,🟢,🏘️ ϟϟ,💦
6,💦,🗻,Scarab 2 ♡♡♡♡,Beetle ♡♡♡♡,🟢,🟢,Scarab ♡♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


In [ ]:
grid = reset_grid3()
v = DFSViewer(grid, dfs)

In [ ]:
v.show(1)

In [10]:
stop

NameError: name 'stop' is not defined

In [ ]:
# grid.show()

In [ ]:
# MOVE (3, 2) to (1, 2)
# CLUSTER SHELL at (6, 2)
# MOVE (5, 3) to (6, 1)
# REGULAR SHELL at (6, 6)
# MOVE (7, 3) to (7, 2)
# BOULDER SHELL at (4, 2)

In [ ]:
grid = reset_grid()
DFSViewer(grid, dfs)

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
stop

In [ ]:
DFS.rate(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
stop

In [ ]:
grid = reset_grid()

In [ ]:
grid.show()

In [ ]:
DFS.rate_dict(grid)

In [ ]:
dfs.show()

In [ ]:
dfs.explored

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
sm.ge

In [ ]:
choice.command.__dict__.copy()

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
ex = Executor()

In [ ]:
ex.execute(siege_mech.gen_actions()[2])

In [ ]:
grid.show()

In [ ]:
ex.execute(siege_mech.gen_actions()[4])

In [ ]:
grid.show()

In [ ]:
ex

In [ ]:
ex.undo()

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
ex = Executor()

score = DFS.rate(grid)

DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[2][0])
score = DFS.rate(grid)

show = grid.show()

dic = DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
score = DFS.rate(grid)

frontier.sort(key=lambda x:x[1], reverse=True)

score = DFS.rate(grid)

ex.execute(frontier[0][0])
score = DFS.rate(grid)

dic = DFS.rate_dict(grid, True)

show = grid.show()

In [ ]:
print(score)

In [ ]:
ex.undo()
score = DFS.rate(grid)

show = grid.show()

ex.execute(frontier[1][0])
score = DFS.rate(grid)

grid.show()

ex.undo()
score = DFS.rate(grid)

grid.show()

ex.execute(frontier[2][0])
score = DFS.rate(grid)

grid.show()

grid.get_tile((3,6)).visitor

grid.get_tile((5,5)).visitor

grid.get_tile((4,6)).type_object.health

grid.get_tile((4,5)).type_object.health

DFS.rate_dict(grid, True)

grid.show()

In [ ]:
print(score)

new score 42 != old score 37 at 
MOVE (7, 3) to (5, 5)
MOVE (3, 2) to (3, 4)
CLUSTER SHELL at (3, 2)
BOULDER SHELL at (5, 1)
MOVE (5, 3) to (3, 3)
HEAL +1

[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, 'UNDO', REGULAR SHELL at (7, 1), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 2), HEAL +1, 'UNDO', REGULAR SHELL at (6, 2), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 3), REGULAR SHELL at (3, 6), 'UNDO', REGULAR SHELL at (3, 7), 'UNDO', HEAL +1]

In [ ]:
stop

In [ ]:
stop

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
grid.show()

In [ ]:
frontier = DFS.gen_rated_frontier(grid)

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    

In [ ]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

In [ ]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))